***Code pour creer une base de donnée en csv puis la déplacer sur MongoDB***

In [ ]:
from dotenv import load_dotenv
import os

# Charger le fichier .env
load_dotenv()

# Accéder aux variables d'environnement
api_key = os.getenv("CLE_GIT")

In [ ]:
from github import Github
import base64
import random
import csv
from markdown import markdown
from bs4 import BeautifulSoup

# Remplacez par votre token d'accès personnel
TOKEN = api_key

# Connexion à l'API GitHub
g = Github(TOKEN)

# Fonction pour récupérer des dépôts aléatoires
def get_random_repositories(github, num_repos=10):
    """Récupère un certain nombre de dépôts publics aléatoires."""
    repositories = []
    search_query = "stars:>1"  # Rechercher des dépôts avec au moins 1 étoile pour une certaine qualité
    for repo in github.search_repositories(query=search_query):
        repositories.append(repo)
        if len(repositories) >= num_repos:
            break
    return repositories

# Fonction pour convertir le Markdown en texte brut
def markdown_to_text(markdown_content):
    # Convertir le Markdown en HTML
    html_content = markdown(markdown_content)
    # Extraire le texte brut du HTML
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()

# Fonction pour supprimer les sauts de ligne multiples
def remove_multiple_newlines(text):
    # Remplace plusieurs sauts de ligne consécutifs par un seul saut de ligne
    return "\n".join(line for line in text.splitlines() if line.strip())

# Fonction pour extraire les informations des dépôts
def extract_repo_data(repo):
    """Extrait les informations d'un dépôt donné."""
    try:
        readme = repo.get_readme()
        readme_content = base64.b64decode(readme.content).decode("utf-8")
    except:
        readme_content = "README non disponible"

    last_commit = repo.get_commits()[0]    
    return {
        "Nom du dépôt": repo.name,
        "Propriétaire": repo.owner.login,
        "Topics":repo.get_topics(),
        "Date de création": repo.created_at,
        "Url": repo.html_url,
        "Date du dernier commit": last_commit.commit.author.date if last_commit else "N/A",
        "Langage principal": repo.language,
        "Nombre d'étoiles": repo.stargazers_count,
        "Description": repo.description or "Pas de description",
        "README": remove_multiple_newlines(markdown_to_text(readme_content))  # Limiter à 1000 caractères
    }

# Récupérer des dépôts aléatoires
num_repos = 10  # Nombre de dépôts à collecter
random_repositories = get_random_repositories(g, num_repos)

# Extraire les informations de chaque dépôt
data = []
for repo in random_repositories:
    repo_data = extract_repo_data(repo)
    data.append(repo_data)
    print(f"Dépôt analysé : {repo_data['Nom du dépôt']}")

# Sauvegarder les données dans un fichier CSV
output_file = "repositories.csv"
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"Données des dépôts sauvegardées dans le fichier : {output_file}")


In [1]:
import csv

# Chemin vers le fichier CSV
csv_file = "repositories.csv"

# Lecture et affichage des données
with open(csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)

['Nom du dépôt', 'Propriétaire', 'Topics', 'Date de création', 'Url', 'Date du dernier commit', 'Langage principal', "Nombre d'étoiles", 'Description', 'README']
['freeCodeCamp', 'freeCodeCamp', "['learn-to-code', 'nonprofits', 'programming', 'nodejs', 'react', 'd3', 'careers', 'education', 'teachers', 'javascript', 'certification', 'curriculum', 'math', 'community', 'freecodecamp', 'hacktoberfest']", '2014-12-24 17:49:19+00:00', 'https://github.com/freeCodeCamp/freeCodeCamp', '2025-01-12 00:49:31+00:00', 'TypeScript', '408786', "freeCodeCamp.org's open-source codebase and curriculum. Learn to code for free.", "freeCodeCamp.org's open-source codebase and curriculum\nfreeCodeCamp.org is a friendly community where you can learn to code for free. It is run by a donor-supported 501(c)(3) charity to help millions of busy adults transition into tech. Our community has already helped more than 40,000 people get their first developer job.\nOur full-stack web development and machine learning cu

In [2]:
from pymongo import MongoClient

# Se connecter au serveur MongoDB
client = MongoClient("mongodb://127.0.0.1:27017")

# Créer ou accéder à une base de données
db = client["my_database"]

# Créer une collection et insérer un document
collection = db["my_collection"]


with open(csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)  # Utiliser DictReader pour lire les lignes comme des dictionnaires
    for row in reader:
        # Insérer chaque ligne du CSV comme un document dans MongoDB
        collection.insert_one(row)

print("Base de données et collection créées avec succès !")


Base de données et collection créées avec succès !
